# Лабораторная 1. Очистка набора данных

Отбор признаков - важная задача перед обучением модели машинного обучения. Признаки отбираются чаще всего вручную на основании описательной статистики, тепловой карты, показывающей линейную корреляцию и важности признаков. Никогда не смотрят только на что-то одно, всегда признаки рассматривают с разных сторон.

# Подключение модулей

Для определения важности признаков можно использовать вектор feature_importance, которые строится с помощью Random Forest. При этом для задач регрессии необходимо подключать именно RandomForestRegressor, для задач классификации - RandomForestClassifier

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor

# Загрузка данных (регрессия)

В данной задаче у нас 28 признаков и 2 целевые переменные (G_total и КГФ). И это вносит дополнительную сложность, так как первая целевая переменная с большим количество пропусков.

In [2]:
df = pd.read_excel('Data.xlsx', sheet_name='VU')

In [3]:
df

,Unnamed: 0,date,Unnamed: 2,Unnamed: 3,осредненные параметры,Unnamed: 5,Unnamed: 6,На конец режима,Unnamed: 8,Unnamed: 9,...,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33
0,Номер,NaT,Глубина манометра,Dшт,Руст,Рзаб,Pлин,Руст,Рзаб,Рлин,...,Рпл. Тек (Карноухов),Pсб,Pсб,Ro_g,Ro_c,Ro_w,Удельная плотность газа,G_total,КГФ,КГФ_1
1,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,804,2008-06-05,3576.3,7.94,249.6,370.1,101.8,249,359.6,101.8,...,-,93.6,92.376018,0.806017,801,1000,0.669449,2.782623,311.9094,NaN
3,804,2008-06-06,3576.3,9.53,233.5,364.6,101.3,231,338.1,102.4,...,-,92.9,91.685171,0.806017,801,1000,0.669449,3.697781,288.6003,NaN
4,804,2008-06-07,3576.3,11.11,213.4,357.1,101.6,211,314.8,100.6,...,-,91.4,90.204787,0.806017,801,1000,0.669449,4.515073,248.7906,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182,22505,2009-09-23,3678,12.7,182.2,271.586,93.255111,179,258.6,92.702141,...,60.8,89.7,88.527017,NaN,NaN,NaN,NaN,NaN,NaN,NaN
183,22505,2009-09-27,3678,14.29,157,249.076,96.472143,153,233.5,100.060417,...,60.8,87.3,86.158401,NaN,NaN,NaN,NaN,NaN,NaN,NaN
184,22505,2010-04-09,3679.5,7.94,211.9,289.3877,91.131226,208,289.6,91.304642,...,61.3,88.2,87.046632,NaN,NaN,NaN,NaN,NaN,NaN,NaN
185,22505,2010-04-14,3679.5,9.53,204,282.8709,91.173008,201,275.3,91.616821,...,61.3,88.5,87.342709,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Описательная статистика

Описательная статистика для непрерывных и категориальных признаках отличается.

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 187 entries, 0 to 186
Data columns (total 34 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Unnamed: 0             186 non-null    object        
 1   date                   185 non-null    datetime64[ns]
 2   Unnamed: 2             186 non-null    object        
 3   Unnamed: 3             186 non-null    object        
 4   осредненные параметры  186 non-null    object        
 5   Unnamed: 5             186 non-null    object        
 6   Unnamed: 6             186 non-null    object        
 7   На конец режима        186 non-null    object        
 8   Unnamed: 8             186 non-null    object        
 9   Unnamed: 9             186 non-null    object        
 10  Unnamed: 10            186 non-null    object        
 11  Unnamed: 11            180 non-null    object        
 12  Unnamed: 12            186 non-null    object        
 13  Unnam

In [5]:
df.describe()

/tmp/ipykernel_11512/3627053830.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  df.describe()


,Unnamed: 0,date,Unnamed: 2,Unnamed: 3,осредненные параметры,Unnamed: 5,Unnamed: 6,На конец режима,Unnamed: 8,Unnamed: 9,...,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33
count,186.0,185,186.0,186.00,186.0,186,186,186.0,186,186,...,183,179.0,179.000000,24.000000,94.0,24.0,94.00,24,24,71.00
unique,15.0,177,40.0,15.00,174.0,180,178,140.0,174,170,...,26,114.0,114.000000,7.000000,26.0,2.0,22.00,24,24,50.00
top,21303.0,2008-02-08 00:00:00,3548.5,9.53,206.0,-,-,215.0,-,-,...,-,89.6,88.428325,0.787416,782.0,1000.0,0.66,G_total,КГФ,0.18
freq,19.0,2,10.0,38.00,3.0,4,7,7.0,4,11,...,47,6.0,6.000000,7.000000,11.0,23.0,9.00,1,1,4.00
first,NaN,2004-09-16 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
last,NaN,2011-10-02 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Стандартные функции pandas не выводят всю необходимую описательную статистику, поэтому создадим свою таблицу с описательной статистикой. Для удобства закрасим признаки с единичной мощностью и большим количеством пропусков, чтобы потом их было легче удалить.

In [6]:
C=len(df.columns)
L=len(df.index)
CN=df.count() #количество
NP=((L-CN)/L)*100 #процент пропущенных значений
MN=df.min() #минимум
Q1=df.quantile(q=0.25) #первый квартиль
MA=df.mean() #среднее значение
ME=df.median() #медиана
Q3=df.quantile(q=0.75) #третий квартиль
MX=df.max() #максимум
ST=df.std() #стандартное отклонение
P=df.nunique() #мощность
IQ=Q3-Q1 #интерквартильный размах

/tmp/ipykernel_11512/3644451440.py:5: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  MN=df.min() #минимум
/tmp/ipykernel_11512/3644451440.py:7: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  MA=df.mean() #среднее значение
/tmp/ipykernel_11512/3644451440.py:7: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  MA=df.mean() #среднее значение
/tmp/ipykernel_11512/3644451440.py:8: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  ME=df.median() #медиана
/tmp/ipykernel_11

In [7]:
frame = pd.concat([CN, NP, MN, Q1, MA, ME, Q3, MX, ST, P, IQ], axis=1, join="inner")
frame=frame.T
f=pd.DataFrame(frame)
f.index=['Количество', 'Процент пропусков', 'Минимум', 'Первый квартиль','Среднее', 'Медиана', 'Третий квартиль', 'Максимум','Стандартное отклонение', 'Мощность', 'Интерквартильный размах']
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
f1=f.style.apply(lambda x: ["background-color:tomato" if i==1 else "" for i in x], axis=1, subset=(["Мощность"], slice(None))).apply(lambda x: ["background-color:tomato" if i>=60 else "" for i in x], axis=1, subset=(["Процент пропусков"], slice(None))).apply(lambda x: ["background-color:steelblue" if (i<30 and i>0) else "" for i in x], axis=1, subset=(["Процент пропусков"], slice(None))).apply(lambda x: ["background-color:limegreen" if i==0 else "" for i in x], axis=1, subset=(["Процент пропусков"], slice(None)))
f1

/home/va1korion/.local/lib/python3.8/site-packages/IPython/core/formatters.py:343: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  return method()

KeyboardInterrupt



Удалим признаки, у которых мощность равна 1 и пропусков слишком много

In [8]:
df=df.drop(['Ro_g', 'Ro_w'], axis=1)

KeyError: "['Ro_g', 'Ro_w'] not found in axis"

# Тепловая карта

In [ ]:
sns.set(rc = {'figure.figsize':(15,8)})
sns.heatmap(df.corr(), annot=True, linewidths=3, cbar=False)

# Распределения

Построим графики распределения значений признаков. Дополнительно покажем линиями первый и третий квартили, медиану и среднее. Также выведем пороги отсечения выбросов по интерквартильному размаху.
Единственный выброс, который здесь можно гарантированно увидеть, это слева у признака "Ro_c"

In [ ]:
for i in df.columns:
    plt.figure(i)
    sns.histplot(df[i],kde=True,stat="density")
    plt.axvline(f.iloc[3][i]-1.5*f.iloc[10]["Dшт"], color="indigo", ls='--') #q1-1.5*iqr
    plt.axvline(f.iloc[3][i], color="dodgerblue", ls='--') #первый квартиль
    plt.axvline(f.iloc[4][i], color="red", ls='--') #среднее
    plt.axvline(f.iloc[5][i], color="goldenrod", ls='--') #медиана
    plt.axvline(f.iloc[6][i], color="dodgerblue", ls='--') #третий квартиль
    plt.axvline(f.iloc[6][i]+1.5*f.iloc[10][i], color="indigo", ls='--') #q3+1.5*iqr
    plt.show()

У признака "Удельная плотность газа" пороги слишком далекие, что лишний раз иллюстрирует, что отсекать через интерквартильный размах можно только при близком к нормальному распределении. Однозначно сказать, что слева наблюдается выброс нельзя из-за малого размера выборки. Это вполне может быть состояние потока со скважины.

In [ ]:
sns.histplot(df[df.columns[25]],kde=True,stat="density")
plt.axvline(f.iloc[3][df.columns[25]], color="dodgerblue", ls='--') #первый квартиль
plt.axvline(f.iloc[4][df.columns[25]], color="red", ls='--') #среднее
plt.axvline(f.iloc[5][df.columns[25]], color="goldenrod", ls='--') #медиана
plt.axvline(f.iloc[6][df.columns[25]], color="dodgerblue", ls='--') #третий квартиль

# Важность признаков

Так как у нас задача регрессии, то можно использовать по построение вектора важности признаков из рандомизированного леса принятия решений. В качестве функции стоимости здесь лучше подходит MSE. sklearn не умеет работать с пропущеными значениями, поэтому для целей определения важности заполним пропуски значением -100, само значение лучше подбирать исходя из распределений признаков с пропусками. Это конечно дает смещение оценки, но в задаче отбора признаков мы этим можем пренебречь. При построении модели регрессии для решения задачи так делать не стоит.

In [ ]:
df1=df.fillna(-100)
X=df1.iloc[:, 0:25]
Y=df1.iloc[:, 26:28]

regr = RandomForestRegressor(max_depth=20, random_state=0)
regr.fit(X, Y)

In [ ]:
feat_importances = pd.Series(regr.feature_importances_, index=X.columns)
feat_importances.plot(kind='barh')

# Выводы

Наша задача оставить признаки, которые помогут в решении задачи.
1) Из трех признаков "Pпл. Тек" можно оставить "Pпл. Тек (послед точка на КВД)". У него меньше всего пропусков, высокая важность и неплохая корреляция с "КГФ". Остальные два удаляем
2) Подозрителен параметр "Dшт"
...